In [1]:
from time import sleep
from getpass import getpass
from itertools import product
from tqdm.contrib.telegram import tqdm

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["text.usetex"] = True

import import_ipynb
import dynamical_system as ds

importing Jupyter notebook from dynamical_system.ipynb


# SIMULATION FOR FIXED PARAMETERS $\alpha$, $\beta$

In [16]:
steps = 10000
sigma = 0
alfa, beta = 2, 4
x0, y0 = 2, 2
model = ds.brusselator(alfa=alfa, beta=beta, sigma=sigma, X0=x0, Y0=y0, t_stop=70, k=steps)

number_simulations = 1
simulation = model.run_simulation(number_simulations)

time_steps, X, Y = simulation
model.plot_parametric(simulation)
model.plot_concentrations(simulation)

### zoomed simulation to the last 50 seconds

In [6]:
stable_start_time = int((steps/7)*2//1)
simulation_cut = time_steps[stable_start_time:], X[:, stable_start_time:], Y[:, stable_start_time:]
model.plot_parametric(simulation_cut)
model.plot_concentrations(simulation_cut)

# SETTING PARAMETERS $\alpha$ AND  $\beta$

#### Given $b_{\text{crit}}=a^2+1$, it is chosen $\beta=b_{crit}-1, b_{crit}, b_{crit}+1$ for any $\alpha = a$.

#### Also we choose a set of positive integer initial values on the square with corners $(0, 0)$ and $(5, 5)$.

In [2]:
alfa = np.linspace(1, 3, 21)
partition = np.linspace(0, 2, 11)  ## 21 ---> 11

beta = []
for a in alfa:
    b = partition + a**2     ## a**2 + 1
    beta.append(b)
beta = np.array(beta)

params = []
for i in range(len(alfa)):
    params.append(list(product([alfa[i]], beta[i])))
    
parameters_unpacked = np.array(params)
parameters = np.concatenate(parameters_unpacked, axis=0)

# print(parameters)
print(len(parameters))

order = 4
mesh = np.zeros((order+1, order+1, 2))
for i in range(order+1):
    for j in range(order+1):
        mesh[i, j] = np.array([i, j])

initial_values = np.flip(np.concatenate(mesh), axis=1)

# print(initial_values)
print(len(initial_values))

231
25


### code snippet to remove initial conditions that match with stability point
### (still not implemented, waiting to address reconstruction precision)

In [17]:
# counter = 0
# for x0, y0 in initial_values:
#     for i, j in parameters:
#         if x0 == i and y0 == j/i:
#             print("a:" , i, "b:", j)
#             print("x:", x0, "y:", y0)
#             print("\n")
#             counter += 1
# print(counter)

# CONFIGURING AND STARTING SIMULATION

In [3]:
number_simulations = 50
steps = 10000

## number of matrices per (initial value, parameter)
number_matrices = 2000

## times where correlation matrices will be taken
start_time = int((steps/70)*20//1)
sampled_times = np.random.choice(np.arange(start_time, steps+1), size=number_matrices, replace=False)

## storage array
correlation_matrices = np.zeros(shape=(number_matrices*len(parameters)*len(initial_values), 2, 2))

## counter to add correlation matrix in the right order
matrix_number = 0

In [4]:
token, chat_id = !pass show programacion/telegram_bot

## simulating the system for each initial condition and parameter pairs
for initial_X, initial_Y in tqdm(initial_values, token=token, chat_id=chat_id):
    data = np.zeros(shape=(len(parameters), 2, number_simulations, steps+1))
    parameter_pair = 0
    for i, j in tqdm(parameters, token=token, chat_id=chat_id):
        model = ds.brusselator(alfa=i, beta=j, sigma=0.05, X0=initial_X, Y0=initial_Y, t_stop=70, k=steps)
        dynamics = model.run_simulation(number_simulations)
        simulations = np.array([dynamics[1], dynamics[2]])
        data[parameter_pair] = simulations
        parameter_pair += 1
        sleep(0.1)

    ## obtaining correlation matrices for each initial condition
    for time in sampled_times:
        for p in range(len(parameters)):
            correlation_matrices[matrix_number] = np.corrcoef(data[p, :, :, time])
            matrix_number += 1

    savetitle = "/home/dynamix/datasets/timeseries_X{}_Y{}".format(initial_X, initial_Y)
    np.savez(savetitle, data)

done = [i for i in tqdm(range(1), token=token, chat_id=chat_id)]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/231 [00:00<?, ?it/s]

  0%|          | 0/231 [00:00<?, ?it/s]

  0%|          | 0/231 [00:00<?, ?it/s]

  0%|          | 0/231 [00:00<?, ?it/s]

...

  0%|          | 0/231 [00:00<?, ?it/s]

  0%|          | 0/231 [00:00<?, ?it/s]

  0%|          | 0/231 [00:00<?, ?it/s]

  0%|          | 0/231 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

# DATA LABELS AND SAVING CORRELATION MATRICES DATABASE

In [5]:
## initial values labels in the order of simulation generation
initial_values_labels = np.zeros(shape=(number_matrices*len(parameters)*len(initial_values), 2))

counter = 0
for initial_value in initial_values:
    for k in range(number_matrices*len(parameters)):
        initial_values_labels[counter] = initial_value
        counter += 1
        
## parameter labels in the order initial values
parameter_labels = np.zeros(shape=(len(initial_values)*number_matrices, len(parameters), 2))

for k in range(len(initial_values)*number_matrices):
    parameter_labels[k] = parameters
    
parameter_labels = np.concatenate(parameter_labels)

In [6]:
print(initial_values_labels.shape)
print(initial_values_labels[:462000+1])
print(parameter_labels.shape)
print(parameter_labels[:462000+1])

(11550000, 2)
[[0. 0.]
 [0. 0.]
 [0. 0.]
 ...
 [0. 0.]
 [0. 0.]
 [1. 0.]]
(11550000, 2)
[[ 1.   1. ]
 [ 1.   1.2]
 [ 1.   1.4]
 ...
 [ 3.  10.8]
 [ 3.  11. ]
 [ 1.   1. ]]


In [8]:
savetitle = "/home/dynamix/datasets/correlations"
np.savez(savetitle, correlation_matrices=correlation_matrices, ab_labels=parameter_labels, xy_labels=initial_values_labels)

In [10]:
correlation_matrices[:2000], parameter_labels[:2000], initial_values_labelsL

array([[[ 1.        , -0.85012452],
        [-0.85012452,  1.        ]],

       [[ 1.        , -0.72690158],
        [-0.72690158,  1.        ]],

       [[ 1.        , -0.77185214],
        [-0.77185214,  1.        ]],

       ...,

       [[ 1.        , -0.91009779],
        [-0.91009779,  1.        ]],

       [[ 1.        , -0.84650716],
        [-0.84650716,  1.        ]],

       [[ 1.        , -0.85564404],
        [-0.85564404,  1.        ]]])